# Store kitti sequence

Create a sequence

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
#os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_0'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_1'

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 21.1 ms, sys: 0 ns, total: 21.1 ms
Wall time: 20.1 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [3]:
seq = Sequence()
for f in ProgressBar()(frame[:20]):
    seq.add_keyframe(f)

100% (20 of 20) |#########################| Elapsed Time: 0:01:17 Time: 0:01:17


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [4]:
len(frame[5].get_observations())

468

In [5]:
mp1 = [o.mappoint for o in frame[1].get_observations() if o.mappoint is not None]
[len(m.observations) for m in mp1]

[1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 2,
 3,
 1,
 3,
 3,
 2,
 2,
 6,
 1,
 2,
 1,
 1,
 2,
 14,
 3,
 1,
 1,
 2,
 1,
 1,
 3,
 2,
 8,
 11,
 2,
 2,
 1,
 3,
 9,
 1,
 2,
 2,
 3,
 2,
 2,
 1,
 1,
 2,
 4,
 2,
 2,
 2,
 4,
 5,
 2,
 2,
 1,
 2,
 2,
 4,
 1,
 1,
 5,
 1,
 2,
 1,
 2,
 4,
 2,
 1,
 1,
 4,
 1,
 5,
 3,
 2,
 3,
 2,
 1,
 1,
 3,
 1,
 3,
 2,
 2,
 3,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 2,
 3,
 4,
 2,
 3,
 1,
 3,
 1,
 1,
 2,
 2,
 3,
 2,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 2,
 8,
 1,
 6,
 4,
 2,
 2,
 2,
 2,
 1,
 2,
 10,
 1,
 2,
 3,
 4,
 5,
 3,
 1,
 3,
 3,
 8,
 2,
 2,
 1,
 1,
 3,
 6,
 3,
 2,
 1,
 1,
 4,
 5,
 2,
 1,
 3,
 4,
 1,
 2,
 1,
 2,
 3,
 2,
 1,
 1,
 3,
 1,
 3,
 1,
 1,
 3,
 3,
 2,
 2,
 2,
 3,
 7,
 2,
 2,
 1,
 1,
 2,
 4,
 1,
 2,
 2,
 1,
 3,
 3,
 4,
 1,
 2,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 3,
 4,
 5,
 2,
 1,
 5,
 1,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 2,
 1,
 2,
 2,
 4,
 2,
 5,
 1,
 2,
 5,
 2,
 2,
 2,
 9,
 4,
 1,
 2,
 3,
 2,
 3,
 1,
 4,
 6,
 2,
 3,
 2,
 4,
 1,
 3,
 1,
 1,
 2,
 3,
 2,
 3,
 1,
 

In [6]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

cv_keyframes = get_covisible_keyframes(seq.keyframes[1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

# np.save('cv_keyframes', cv_keyframes_BA)
# np.save('f_keyframes', f_keyframes_np)
# np.save('mappoints', mappoints_np)
# np.save('links', links_np)


Keyframes count
 20 

mappoints count  
 (1519, '\n')
mappoints 
 [[  3.66300000e+03  -5.56711489e+00  -3.73793304e+00   1.75375995e+01
    1.00000000e+00]
 [  1.00300000e+03  -6.64412977e+00   4.35717484e-01   2.27228171e+01
    1.00000000e+00]
 [  1.00400000e+03   1.15492369e+01  -9.09458131e-01   1.96825427e+01
    1.00000000e+00]
 ..., 
 [  3.19700000e+03  -1.14568556e+01  -1.81989205e+00   2.28650583e+01
    1.00000000e+00]
 [  3.19900000e+03  -1.19343114e+01  -1.00161262e+00   2.55182484e+01
    1.00000000e+00]
 [  3.20000000e+03   3.76567953e+00  -5.22260437e-01   5.93887355e+00
    1.00000000e+00]] 

links count  
 (4668, '\n')
links 
 [[  3.66300000e+03   1.30000000e+01   3.79000000e+02   3.20000000e+01]
 [  3.66300000e+03   1.20000000e+01   3.88000000e+02   3.80000000e+01]
 [  1.00300000e+03   3.00000000e+00   3.99000000e+02   1.98000000e+02]
 ..., 
 [  3.19900000e+03   1.30000000e+01   2.71000000e+02   1.57000000e+02]
 [  3.20000000e+03   1.20000000e+01   1.06300000e+03   1.

14 

Keyframes after BA 
 [[  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  9.99989951e-01   2.88687753e-03   3.42995166e-03   1.02907941e-03
   -2.89330954e-03   9.99994063e-01   1.87176716e-03   8.38253233e-03
   -3.42452774e-03  -1.88167226e-03   9.99992366e-01  -6.68973946e-01
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  9.99954977e-01   6.57762341e-03   6.83954622e-03   2.98031758e-02
   -6.59502573e-03   9.99975065e-01   2.52493359e-03   2.08158141e-02
   -6.82276762e-03  -2.56992689e-03   9.99973422e-01  -1.19375048e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  9.98795675e-01   4.79906355e-02   1.02028838e-02  -1.30020485e-01
   -4.80061757e-02   9.98846214e-01   1.28356949e-03  -4.0354

In [6]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.cumulative_pose.flatten() for kf in keyframes ] ])

for i in range(len(seq.keyframes)):
    f = seq.keyframes[i]
    pose = f.get_pose()
    if i > 0:
        f.cumulative_pose = np.dot(seq.keyframes[i-1].cumulative_pose, pose)
    else:
        f.cumulative_pose = pose

cv_keyframes = get_covisible_keyframes(seq.keyframes[1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

# np.save('cv_keyframes', cv_keyframes_BA)
# np.save('f_keyframes', f_keyframes_np)
# np.save('mappoints', mappoints_np)
# np.save('links', links_np)


Keyframes count
 20 

mappoints count  
 (1519, '\n')
mappoints 
 [[  2.46100000e+03   3.52533337e+00  -1.04377851e-01   6.68996535e+00
    1.00000000e+00]
 [  1.67000000e+03  -9.79119350e+00  -5.63984439e+00   1.29878106e+02
    1.00000000e+00]
 [  2.99500000e+03  -6.44451494e+00  -3.43479395e+00   1.88172775e+01
    1.00000000e+00]
 ..., 
 [  3.51000000e+03  -1.14605704e+01  -6.54138504e-02   2.12227011e+01
    1.00000000e+00]
 [  2.63900000e+03  -6.70251704e+00  -3.05234987e+00   1.88804096e+01
    1.00000000e+00]
 [  2.27400000e+03  -1.19936217e+01  -7.42976100e-01   2.77946069e+01
    1.00000000e+00]] 

links count  
 (4668, '\n')
links 
 [[ 2461.    11.   986.   174.]
 [ 2461.    10.   942.   174.]
 [ 2461.     8.   876.   180.]
 ..., 
 [ 2639.    11.   352.    69.]
 [ 2274.     8.   297.   166.]
 [ 2274.     7.   301.   168.]] 

Keyframes 
 [<src.frame.Frame object at 0x7f995c070710>, <src.frame.Frame object at 0x7f995c070240>, <src.frame.Frame object at 0x7f995c070208>, <src.fr


14 

Keyframes after BA 
 [[  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  9.99989951e-01   2.88687753e-03   3.42995166e-03   1.02907941e-03
   -2.89330954e-03   9.99994063e-01   1.87176716e-03   8.38253233e-03
   -3.42452774e-03  -1.88167226e-03   9.99992366e-01  -6.68973946e-01
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  9.99967498e-01   5.50042262e-03   5.89470081e-03   1.09016408e-01
   -5.51407395e-03   9.99982147e-01   2.30211921e-03   5.06712977e-02
   -5.88193294e-03  -2.33454820e-03   9.99979976e-01  -1.46591679e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  9.99930453e-01   6.24036390e-03   1.00073354e-02   9.53338560e-02
   -6.27467607e-03   9.99974531e-01   3.40097798e-03   5.397

In [11]:


for i in range(len(seq.keyframes) - 1, 0, -1):
    f = seq.keyframes[i]
    prev = seq.keyframes[i-1]
    previnv = np.linalg.pinv(prev.cumulative_pose)
    pose = np.dot(previnv, f.cumulative_pose)
    print(i, f.get_pose(), '\n', pose)

19 [[  9.99998873e-01  -6.65329159e-04   1.34618546e-03   2.67496501e-03]
 [  6.65103157e-04   9.99999765e-01   1.68324665e-04   4.83976160e-04]
 [ -1.34629714e-03  -1.67429123e-04   9.99999080e-01  -8.85944695e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99998873e-01  -6.65329159e-04   1.34618546e-03   2.67496501e-03]
 [  6.65103157e-04   9.99999765e-01   1.68324665e-04   4.83976160e-04]
 [ -1.34629714e-03  -1.67429123e-04   9.99999080e-01  -8.85944695e-01]
 [  1.54644229e-16  -5.64069887e-15  -1.51177153e-16   1.00000000e+00]]
18 [[ 0.99998751 -0.00456045  0.00204665  0.00485854]
 [ 0.00455577  0.99998701  0.00228676 -0.00186741]
 [-0.00205705 -0.00227741  0.99999529 -0.87978548]
 [ 0.          0.          0.          1.        ]] 
 [[  9.99987507e-01  -4.56045454e-03   2.04665176e-03   4.85854300e-03]
 [  4.55577201e-03   9.99987008e-01   2.28675855e-03  -1.86740754e-03]
 [ -2.05705383e-03  -2.27740590e-03   9.99995291e-01  -8.79785478e-01]
 [